# ***Installation and Importing required packages***

In [1]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
import torch

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
print(device)
torch_dtype

cuda:0


torch.float16

In [4]:
# Load model directly
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
model.to(device) # Moving model to GPU

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

# ***Speech to text***

In [5]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128, # Large v3 using 128 instead of 80 in lower model Large v2, medium etc..
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
#audio = "Deep Learning Course in Arabic.mp3"
audio = "Arabic Audio.mp3"

In [ ]:
%%timeit
result = pipe(audio, generate_kwargs={"language": "arabic"}) # v2 gives better results if you don't provide a language.

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `

26 s ± 279 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
result = pipe(audio, generate_kwargs={"language": "arabic"})

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=arabic, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=arabic.


In [ ]:
result['text']

' السلام عليكم و رحمة الله و بركاته اللهم أنت من فانا فانا بما لمت أنك أنت العز الحكيم سأتكلم في الدورة عن التعلم العميق وسأشرح المواضيع المختصة بالتعلم العميق من ناحية نظرية وعملية طيب انت ممكن عمرك اشتغلت على مشروع خاص بالتعلم العميق وممكن شفت انه اوك انت اخي باستخدام 4 اكواد او 4 صور من الأكواد تستطيع أن تصنع شبكة عصبية لكن لو أخبرتك ما مخرجات الطبقة الفلانية ولماذا حجم الطبقة التي بعد بعدها كان كذا مثلاً الكثير ما يقدر أو ما يقدر يغير أصلاً الحجم لأنه حيضيع ومحايعرف كيف يخلي الحجم مناسب للطبقة اللي هو يستخدمها هذا مجرد موضوع واحد من مواضيع كثيرة فمهم أنه نفهم أو نوازن بين المفهوم النظري والعملي بحيث أنه نتكلم عن جانب نظري بأول شيء وبعدين نشوف الجانب العملي بأخر شيء ونقارن بين الشيء اللي تعلمناه في نفس الوقت هذا اللي حركناه في نفس الوقت هذا اللي حركز عليه في الدورة بحيث أنه أنا أشرح الجانب النظري والجانب العملي طبعا الدورة هتكون عن طريق فيديوهات مثل ما نشوفه الحين وإيضا هتكون عن طريق مقالات وحنشوف كيف نوصل للمقالات والفيديوهات فأنت اختار الطريقة اللي تناسبك تتعلم عن طريقها طيب الشي 

In [ ]:
result['chunks']

[{'timestamp': (0.0, 2.0), 'text': ' السلام عليكم و رحمة الله و بركاته'},
 {'timestamp': (2.0, 5.0),
  'text': ' اللهم أنت من فانا فانا بما لمت أنك أنت العز الحكيم'},
 {'timestamp': (5.0, 8.0), 'text': ' سأتكلم في الدورة عن التعلم العميق'},
 {'timestamp': (8.0, 12.0), 'text': ' وسأشرح المواضيع المختصة بالتعلم العميق'},
 {'timestamp': (12.0, 13.82), 'text': ' من ناحية نظرية وعملية'},
 {'timestamp': (13.82, 18.56),
  'text': ' طيب انت ممكن عمرك اشتغلت على مشروع خاص بالتعلم العميق'},
 {'timestamp': (18.56, 22.48),
  'text': ' وممكن شفت انه اوك انت اخي باستخدام 4 اكواد'},
 {'timestamp': (22.48, 24.0), 'text': ' او 4 صور من الأكواد'},
 {'timestamp': (24.0, 26.0), 'text': ' تستطيع أن تصنع شبكة عصبية'},
 {'timestamp': (26.0, 28.0), 'text': ' لكن لو أخبرتك'},
 {'timestamp': (28.0, 30.0), 'text': ' ما مخرجات الطبقة الفلانية'},
 {'timestamp': (30.0, 32.0), 'text': ' ولماذا'},
 {'timestamp': (32.0, 44.46),
  'text': ' حجم الطبقة التي بعد بعدها كان كذا مثلاً الكثير ما يقدر أو ما يقدر يغير أصلاً ال

In [ ]:
# translation

# result = pipe(audio,
#               return_timestamps=True,
#               generate_kwargs={"language": "french", "task": "translate"})


# print(result["chunks"])

# ***Gradio Interface***

In [6]:
!pip install gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 98.1 MB/s eta 0:00:00


In [7]:
import gradio as gr
import time

In [8]:
def transcribe(audio):

    result = pipe(audio)
    return result['text']

In [ ]:
transcribe(audio)

In [9]:
gr.Interface(
    title = 'Sppech to Text using Whisper Large-v3',
    fn=transcribe,
    inputs=gr.Audio(type="filepath"),
    outputs=["textbox"],
    live=True).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a6b8e4539c19f6530e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# ***Distill Whisper***

In [ ]:
model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True#, use_flash_attention_2=True
)
model.to(device)

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
result = pipe(audio)

In [ ]:
result